In [ ]:
# Import required libraries
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# Download and load dataset
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

# Load the dataset
train_data = pd.read_csv("train-data.tsv", sep="\t", header=None, names=["label", "message"])
test_data = pd.read_csv("valid-data.tsv", sep="\t", header=None, names=["label", "message"])

# Convert labels: "ham" → 0, "spam" → 1
train_data["label"] = train_data["label"].map({"ham": 0, "spam": 1})
test_data["label"] = test_data["label"].map({"ham": 0, "spam": 1})

# Tokenization settings
MAX_WORDS = 6000  # Increased vocabulary size
MAX_LENGTH = 150  # Increased sequence length

# Tokenizer setup
tokenizer = Tokenizer(num_words=MAX_WORDS, oov_token="<OOV>")
tokenizer.fit_on_texts(train_data["message"])

# Convert text to sequences
X_train = tokenizer.texts_to_sequences(train_data["message"])
X_test = tokenizer.texts_to_sequences(test_data["message"])

# Pad sequences
X_train = pad_sequences(X_train, maxlen=MAX_LENGTH, padding="post")
X_test = pad_sequences(X_test, maxlen=MAX_LENGTH, padding="post")

# Convert labels to numpy arrays
y_train = np.array(train_data["label"])
y_test = np.array(test_data["label"])

# Define the improved neural network model
model = keras.Sequential([
    keras.layers.Embedding(MAX_WORDS, 32, input_length=MAX_LENGTH),
    keras.layers.Conv1D(32, 5, activation="relu"),
    keras.layers.MaxPooling1D(pool_size=2),
    keras.layers.LSTM(32, return_sequences=True),
    keras.layers.GlobalAveragePooling1D(),
    keras.layers.Dense(16, activation="relu"),
    keras.layers.Dropout(0.3),  # Added dropout to prevent overfitting
    keras.layers.Dense(1, activation="sigmoid")  # Sigmoid for binary classification
])

# Compile the model
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
model.fit(X_train, y_train, epochs=15, batch_size=32, validation_data=(X_test, y_test), verbose=1)

# Function to predict messages
def predict_message(pred_text):
    seq = tokenizer.texts_to_sequences([pred_text])
    padded_seq = pad_sequences(seq, maxlen=MAX_LENGTH, padding="post")

    prediction_prob = model.predict(padded_seq)[0][0]
    label = "spam" if prediction_prob > 0.45 else "ham"  # Adjusted threshold to 0.45

    return [prediction_prob, label]

# Run test predictions
def test_predictions():
    test_messages = [
        "how are you doing today",
        "sale today! to stop texts call 98912460324",
        "i dont want to go. can we try it a different day? available sat",
        "our new mobile video service is live. just install on your phone to start watching.",
        "you have won £1000 cash! call to claim your prize.",
        "i'll bring it tomorrow. don't forget the milk.",
        "wow, is your arm alright. that happened to me one time too"
    ]

    test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
    passed = True

    for msg, ans in zip(test_messages, test_answers):
        prediction = predict_message(msg)
        print(f"Message: {msg} | Prediction: {prediction}")
        if prediction[1] != ans:
            passed = False

    if passed:
        print("✅ You passed the challenge. Great job!")
    else:
        print("❌ You haven't passed yet. Keep trying.")

# Run the test function
test_predictions()
